In [1]:
import polars as pl

In [2]:
!gsutil cp /home/eanegrin/buckets/b1/datasets/competencia_03_inflacion_adj_ipc.parquet /home/eanegrin/datasets/

Copying file:///home/eanegrin/buckets/b1/datasets/competencia_03_inflacion_adj_ipc.parquet...
- [1 files][861.8 MiB/861.8 MiB]                                                
Operation completed over 1 objects/861.8 MiB.                                    


In [3]:
base_path = '/home/eanegrin/'
# base_path = 'C:/Eugenio/Maestria/DMEyF/'

dataset_path = base_path + 'datasets/'

dataset_file = 'competencia_03_inflacion_adj_ipc.parquet'

In [9]:
df = pl.read_parquet(dataset_path + dataset_file)

In [10]:
# corrijo el tipo de estas 2:

df = df.with_columns([
    pl.col("tmobile_app").cast(pl.Int64),
    pl.col("cmobile_app_trx").cast(pl.Int64)
])

In [11]:
# elimino columnas con data drifting
rotas = ['cmobile_app_trx','tmobile_app','active_quarter','mplazo_fijo_pesos','minversion2']
df = df.select(pl.exclude(rotas))

In [ ]:
# elimino 202006 por variables rotas
df = df.filter(~pl.col("foto_mes").is_in(['202006']]))

In [12]:
# lista de columnas originales para operar despues
originales = df.select(pl.exclude(["numero_de_cliente", "foto_mes", "clase_ternaria"])).columns

In [ ]:
# suma de columnas
df = df.with_columns([
    (pl.col("mcuenta_corriente_adicional").fill_null(0) +
     pl.col("mcuenta_corriente").fill_null(0) +
     pl.col("mcaja_ahorro_adicional").fill_null(0) +
     pl.col("mcaja_ahorro_dolares").fill_null(0) +
     pl.col("mcaja_ahorro").fill_null(0)).alias("suma_alt"),
    
    (pl.col("cseguro_vida").fill_null(0) +
     pl.col("cseguro_auto").fill_null(0) +
     pl.col("cseguro_vivienda").fill_null(0) +
     pl.col("cseguro_accidentes_personales").fill_null(0)).alias("cseguro_todos"),
    
    (pl.col("matm").fill_null(0) +
     pl.col("matm_other").fill_null(0)).alias("matm_total"),
    
    (pl.col("Visa_msaldototal").fill_null(0) +
     pl.col("Master_msaldototal").fill_null(0)).alias("tarjeta_msaldototal"),
    
    (pl.col("mactivos_margen").fill_null(0) +
     pl.col("mpasivos_margen").fill_null(0)).alias("map_margen"),
    
    (pl.col("ccuenta_corriente").fill_null(0) +
     pl.col("ccaja_ahorro").fill_null(0)).alias("cc_ca"),
    
    (pl.col("ctarjeta_visa").fill_null(0) +
     pl.col("ctarjeta_master").fill_null(0)).alias("ctarjeta_total"),
    
    (pl.col("ctarjeta_visa_transacciones").fill_null(0) +
     pl.col("ctarjeta_master_transacciones").fill_null(0)).alias("ctarjeta_total_transacciones"),
    
    (pl.col("mtarjeta_visa_consumo").fill_null(0) +
     pl.col("mtarjeta_master_consumo").fill_null(0)).alias("mtarjeta_total_consumo"),
    
    (pl.col("minversion1_pesos").fill_null(0) +
     pl.col("minversion1_dolares").fill_null(0)).alias("minversion1"),
    
    (pl.col("mpayroll").fill_null(0) +
     pl.col("mpayroll2").fill_null(0)).alias("mpayroll_todos"),
    
    (pl.col("ctarjeta_visa_debitos_automaticos").fill_null(0) +
     pl.col("ctarjeta_master_debitos_automaticos").fill_null(0)).alias("ctarjeta_todas_debitos_automaticos"),
    
    (pl.col("mttarjeta_visa_debitos_automaticos").fill_null(0) +
     pl.col("mttarjeta_master_debitos_automaticos").fill_null(0)).alias("mtarjeta_todas_debitos_automaticos"),
    
    (pl.col("mcomisiones_mantenimiento").fill_null(0) +
     pl.col("mcomisiones_otras").fill_null(0)).alias("mcomisiones_suma"),
    
    (pl.col("Master_cconsumos").fill_null(0) +
     pl.col("Visa_cconsumos").fill_null(0)).alias("tarjetas_cconsumos"),
    
    (pl.col("Master_mfinanciacion_limite").fill_null(0) +
     pl.col("Visa_mfinanciacion_limite").fill_null(0)).alias("tarjetas_mfinanciacion_limite")
])

df = df.with_columns([
    (pl.col("mcuenta_debitos_automaticos").fill_null(0) +
     pl.col("mtarjeta_total_consumo").fill_null(0)).alias("consumos_mas_DA")
])

In [ ]:
# columnas creadas en el paso anterior
creadas = [
    "suma_alt", "cseguro_todos", "matm_total", "tarjeta_msaldototal",
    "map_margen", "cc_ca", "ctarjeta_total", "ctarjeta_total_transacciones",
    "mtarjeta_total_consumo", "minversion1", "mpayroll_todos", "ctarjeta_todas_debitos_automaticos",
    "mtarjeta_todas_debitos_automaticos", "mcomisiones_suma", "tarjetas_cconsumos", "tarjetas_mfinanciacion_limite",
    "consumos_mas_DA"
]

todas = originales + creadas

In [ ]:
# lag 1
df = df.with_columns([
    pl.col(col)
    .sort_by(pl.col("foto_mes")) 
    .shift(1)
    .over("numero_de_cliente") 
    .alias(f"{col}_lag1")
    for col in todas
])

# delta 1
df = df.with_columns([
    (pl.col(col) - pl.col(f"{col}_lag1")).alias(f"{col}_delta1")
    for col in todas
])

In [ ]:
# moving average de los meses -2,-3 y -4
df = df.with_columns([
    pl.col(col)
    .sort_by(pl.col("foto_mes"))
    .shift(2)
    .rolling_mean(window_size=3, min_periods=1)
    .over("numero_de_cliente")
    .alias(f"{col}_avg_lag_234")
    for col in todas
])

# delta
df = df.with_columns([
    (pl.col(col) - pl.col(f"{col}_avg_lag_234")).alias(f"{col}_delta_lag_234")
    for col in todas
])

In [ ]:
# moving average de los meses -5,-6 y -7
df = df.with_columns([
    pl.col(col)
    .sort_by(pl.col("foto_mes"))
    .shift(5)
    .rolling_mean(window_size=3, min_periods=1)
    .over("numero_de_cliente")
    .alias(f"{col}_avg_lag_567")
    for col in todas
])

# delta
df = df.with_columns([
    (pl.col(col) - pl.col(f"{col}_avg_lag_567")).alias(f"{col}_delta_lag_567")
    for col in todas
])

In [ ]:
# agrego tambien el lag 10,11 y 12 y los delta para las importantes

importantes = ['ctrx_quarter',
 'mcuentas_saldo',
 'mrentabilidad_annual',
 'mpasivos_margen',
 'mcaja_ahorro',
 'mcuenta_corriente',
 'mrentabilidad',
 'mactivos_margen',
 'Visa_mfinanciacion_limite',
 'Master_mfinanciacion_limite',
 'chomebanking_transacciones',
 'Visa_Fvencimiento',
 'Master_Fvencimiento',
 'mprestamos_personales',
 'mcomisiones',
 'mcuentas_saldo_delta6',
 'Visa_mconsumospesos',
 'Master_mconsumospesos',
 'mcomisiones_mantenimiento',
 'Visa_msaldototal',
 'Master_msaldototal']

df = df.with_columns([
    pl.col(col)
    .sort_by(pl.col("foto_mes"))
    .shift(10)
    .rolling_mean(window_size=3, min_periods=1)
    .over("numero_de_cliente")
    .alias(f"{col}_avg_lag_101112")
    for col in importantes
])

# delta
df = df.with_columns([
    (pl.col(col) - pl.col(f"{col}_avg_lag_101112")).alias(f"{col}_delta_lag_101112")
    for col in importantes
])

In [15]:
dataset_path

'/home/eanegrin/datasets/'

In [16]:
df.write_parquet(dataset_path + 'competencia_03_fe_v07.parquet')

In [17]:
!gsutil cp /home/eanegrin/datasets/competencia_03_fe_v07.parquet /home/eanegrin/buckets/b1/datasets/

Copying file:///home/eanegrin/datasets/competencia_03_fe_v07.parquet...
- [1 files][  5.7 GiB/  5.7 GiB]                                                
Operation completed over 1 objects/5.7 GiB.                                      


In [18]:
# Dataset con solo periodo de test y de prediccion para que nos de la RAM para levantarlo localmente
df = df.filter(pl.col("foto_mes").is_in([202107,202109]))

df.write_parquet(dataset_path + 'competencia_03_fe_v07_predict.parquet')

In [19]:
!gsutil cp /home/eanegrin/datasets/competencia_03_fe_v07_predict.parquet /home/eanegrin/buckets/b1/datasets/

Copying file:///home/eanegrin/datasets/competencia_03_fe_v07_predict.parquet...
- [1 files][440.5 MiB/440.5 MiB]                                                
Operation completed over 1 objects/440.5 MiB.                                    
